In [1]:
# Script to scrape data to create a bot that can write in the distinctive styles of 2/3 of the Puck Soup Podcast
# as well as Emily Kaplan from ESPN on Ice
# If Sean's content wasn't behind a paywall, he could have a bot too

import requests, pandas, numpy, traceback
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from textgenrnn import textgenrnn


# These functions parse the written content dependent upon the domain it's coming from

def domain_specific_parser(url):

    # Determine domain
    result = urlparse(url)
    domain = result.netloc
    
    # Check domain before downloading url for performance
    valid_domains = ["www.collegehockeynews.com",
                     "sports.yahoo.com",
                     "www.espn.com",
                     "www.si.com",
                     "www.foxsports.com",
                     "sports.yahoo.com",
                     "ca.sports.yahoo.com"]
    if domain not in valid_domains:
        print("Unhandled domain: {}".format(domain))
        return ""
    
    # Get and soupify html
    response = requests.get(url)
    if response.status_code != 200:
        print("HTTP status code {} raised for url: {}".format(response.status_code, url))
        return ""
    html = response.text
    soup = BeautifulSoup(html)
    try:
        if domain == "www.collegehockeynews.com":
            parent_element = soup.find("div", {"id":"content"})
            actual_text = parent_element.findAll("p",{"class":None})
            actual_text = [x if len(x.findChildren()) == 0 else None for x in actual_text]
        elif domain == "sports.yahoo.com":
            actual_text = soup.findAll("p")
        elif domain.startswith("abc"):
            actual_text = soup.findAll("div",{"class":"body-text"})
        elif domain == "www.espn.com":
            text_parent = soup.find("div", {"class":"article-body"})
            actual_text = text_parent.findAll("p")
        elif domain == "www.si.com":
            text_parent = soup.find("div", {"id":"article-body"})
            actual_text = text_parent.findAll("p")
            actual_text = [x if len(x.findChildren()) == 0 else None for x in actual_text]
        elif domain == "www.foxsports.com":
            text_parent = soup.find("div", {"class": "flex-article-content"})
            actual_text = text_parent.findAll("p")
        elif domain == "sports.yahoo.com" or domain == "ca.sports.yahoo.com":
            text_parent = soup.find("div", {"class": "canvas-body"})
            actual_text = text_parent.findAll("p")
        else:
            print("Unhandled domain: {}".format(domain))
            return ""
    except:
        print("Unhandled exception: {}".format(traceback.print_exc()))
        return ""
    text_content = [x.text if x else "" for x in actual_text]
    text_content = "\n".join(text_content)
    return text_content

# Testing for the parsers
# url = "https://www.foxsports.com/nfl/story/nfl-draft-2017-mitch-trubisky-north-carolina-tar-heels-quarterback-122116"
# url = "https://sports.yahoo.com/charlie-coyle-just-beginning-bruins-191401268.html"
# url = "https://abc7ny.com/sports/nhl-power-rankings-reasons-for-optimism-for-all-31-teams/5173856/"
# url = "https://www.si.com/2013/11/05/nick-foles-record-seven-touchdowns-photos"
# url = "https://sports.yahoo.com/blogs/nhl-puck-daddy/on-sidney-crosby--nhl-all-star-game-suspensions-and-bad-vibes-220351008.html"


# text = domain_specific_parser(url)

# print(text)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [77]:
# Function for parsing the links and titles
def get_links(html, writer_id):
    
    soup = BeautifulSoup(html)
    stories = soup.findAll("div", {"class":"news-story"})
    titles_links = [{"title": x.find("h4").find("a").text, "link": x.find("h4").find("a")["href"], "writer":writer_id} for x in stories]
    return titles_links

# List of articles, with page number specs
ryan = "https://muckrack.com/ryan-lambert/articles?page="
greg = "https://muckrack.com/greg-wyshynski/articles?page="
emily = "https://muckrack.com/emily-kaplan/articles?page="
ryan_pagecount = 41
greg_pagecount = 262
emily_pagecount = 15


In [78]:
# Grab links for Ryan
ryan_links = []
for pg in range(1,ryan_pagecount+1):
    print("Processing Ryan, page {} / {}".format(pg,ryan_pagecount))
    html = requests.get(ryan + str(pg)).text
    titles_links = get_links(html, "ryan")
    ryan_links = ryan_links + titles_links

# Grab links for Greg
greg_links = []
for pg in range(1,greg_pagecount+1):
    print("Processing Greg, page {} / {}".format(pg,greg_pagecount))
    html = requests.get(greg + str(pg)).text
    titles_links = get_links(html,"greg")
    greg_links = greg_links + titles_links

# Grab links for Emily
emily_links = []
for pg in range(1,emily_pagecount+1):
    print("Processing Emily, page {} / {}".format(pg,emily_pagecount))
    html = requests.get(emily + str(pg)).text
    titles_links = get_links(html,"emily")
    emily_links = emily_links + titles_links    


Processing Ryan, page 1 / 41
Processing Ryan, page 2 / 41
Processing Ryan, page 3 / 41
Processing Ryan, page 4 / 41
Processing Ryan, page 5 / 41
Processing Ryan, page 6 / 41
Processing Ryan, page 7 / 41
Processing Ryan, page 8 / 41
Processing Ryan, page 9 / 41
Processing Ryan, page 10 / 41
Processing Ryan, page 11 / 41
Processing Ryan, page 12 / 41
Processing Ryan, page 13 / 41
Processing Ryan, page 14 / 41
Processing Ryan, page 15 / 41
Processing Ryan, page 16 / 41
Processing Ryan, page 17 / 41
Processing Ryan, page 18 / 41
Processing Ryan, page 19 / 41
Processing Ryan, page 20 / 41
Processing Ryan, page 21 / 41
Processing Ryan, page 22 / 41
Processing Ryan, page 23 / 41
Processing Ryan, page 24 / 41
Processing Ryan, page 25 / 41
Processing Ryan, page 26 / 41
Processing Ryan, page 27 / 41
Processing Ryan, page 28 / 41
Processing Ryan, page 29 / 41
Processing Ryan, page 30 / 41
Processing Ryan, page 31 / 41
Processing Ryan, page 32 / 41
Processing Ryan, page 33 / 41
Processing Ryan, pa

Processing Greg, page 223 / 262
Processing Greg, page 224 / 262
Processing Greg, page 225 / 262
Processing Greg, page 226 / 262
Processing Greg, page 227 / 262
Processing Greg, page 228 / 262
Processing Greg, page 229 / 262
Processing Greg, page 230 / 262
Processing Greg, page 231 / 262
Processing Greg, page 232 / 262
Processing Greg, page 233 / 262
Processing Greg, page 234 / 262
Processing Greg, page 235 / 262
Processing Greg, page 236 / 262
Processing Greg, page 237 / 262
Processing Greg, page 238 / 262
Processing Greg, page 239 / 262
Processing Greg, page 240 / 262
Processing Greg, page 241 / 262
Processing Greg, page 242 / 262
Processing Greg, page 243 / 262
Processing Greg, page 244 / 262
Processing Greg, page 245 / 262
Processing Greg, page 246 / 262
Processing Greg, page 247 / 262
Processing Greg, page 248 / 262
Processing Greg, page 249 / 262
Processing Greg, page 250 / 262
Processing Greg, page 251 / 262
Processing Greg, page 252 / 262
Processing Greg, page 253 / 262
Processi

In [142]:
# combine list of links
all_links = greg_links + ryan_links + emily_links
df = pandas.DataFrame.from_records(all_links)

# De-dupe
df = df.drop_duplicates(subset="title", keep="last")
df["freetext"] = ""
df["error"] = ""
df["domain"] = [urlparse(url).netloc for url in df["link"].values]
df.reset_index(drop=True, inplace=True)

# Get rid of Greg and Emily's pesky paywall stuff
ix = ((df["writer"] == "greg") | (df["writer"] == "emily")) & (numpy.array(["/insider/" in x for x in df["link"].values]))
ix = numpy.logical_not(ix)
df = df[ix]
df.reset_index(inplace=True, drop=True)

# Seriously? Some of these links and titles have newline characters in them. Replace them.
df["title"] = [x.replace("\n"," ") for x in df["title"]]
df["link"] = [x.replace("\n","") for x in df["link"]]

# write to file
df.to_csv("/media/sf_VBoxShared/link_list.csv", index=False, sep="|")


In [220]:
# Read in file
df = pandas.read_csv("/media/sf_VBoxShared/link_list.csv",sep="|")
# Generate counts
a = df.groupby("writer")["link"].count()
print("Article counts per author")
print(a)
# Get a list of unique domains per writer
df_domains = df.copy()
df_domains["domain"] = [urlparse(url).netloc for url in df_domains["link"].values]
df_domains = df_domains.drop_duplicates(subset=["writer","domain"], keep="last").reset_index(drop=True)
diff_domains = df_domains.groupby("writer")["domain"].count()
print("Different domains per author")
print(diff_domains)


Article counts per author
writer
emily     250
greg     6050
ryan      989
Name: link, dtype: int64
Different domains per author
writer
emily    12
greg     28
ryan      8
Name: domain, dtype: int64


In [221]:
# Iterate over links, grabbing data
for ix, row in df.iterrows():
    print("Processing {} / {} records\nurl: {}".format(ix,len(df), row["link"]))
    if not (row["freetext"] == "" or pandas.isnull(row["freetext"])):
        print("text already downloaded, skipping...")
        continue
    text = domain_specific_parser(row["link"])
    df.loc[ix,"freetext"] = text
    df.to_csv("/media/sf_VBoxShared/link_list.csv", index=False, sep="|")

Processing 0 / 7289 records
url: https://abc7news.com/sports/nhl-power-rankings-the-hidden-mvp-for-all-31-teams/5221962/
text already downloaded, skipping...
Processing 1 / 7289 records
url: https://abc30.com/sports/stanley-cup-madness-who-was-the-best-champ-of-the-past-16/5217860/
text already downloaded, skipping...
Processing 2 / 7289 records
url: https://abc7.com/sports/stanley-cup-madness-who-was-the-best-champ-of-the-last-16/5217860/
text already downloaded, skipping...
Processing 3 / 7289 records
url: https://abc11.com/sports/justin-faulks-long-journey-to-the-nhl-playoffs/5211527/
text already downloaded, skipping...
Processing 4 / 7289 records
url: https://abc7chicago.com/sports/nhl-power-rankings-every-teams-college-hoops-equivalent/5209740/
text already downloaded, skipping...
Processing 5 / 7289 records
url: https://abc7ny.com/sports/nhl-power-rankings-the-puck-luck-edition/5191701/
text already downloaded, skipping...
Processing 6 / 7289 records
url: https://6abc.com/sports

Processing 54 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25510562/stars-rookie-miro-heiskanen-living-texas-sized-expectations
text already downloaded, skipping...
Processing 55 / 7289 records
url: http://www.wohfradio.com/national-sports/old-man-ovechkin-is-still-learning-new-tricks/
Unhandled domain: www.wohfradio.com
Processing 56 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25504034/old-man-alex-ovechkin-learning-new-tricks
text already downloaded, skipping...
Processing 57 / 7289 records
url: http://www.espn.com/espn/now?nowId=21-41048205-4
text already downloaded, skipping...
Processing 58 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25476333/why-how-nhl-expand-playoffs
text already downloaded, skipping...
Processing 59 / 7289 records
url: http://www.wohfradio.com/national-sports/knights-signed-pics-of-reaves-hit-destroyed/
Unhandled domain: www.wohfradio.com
Processing 60 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25474313/ryan-reaves-a

Processing 111 / 7289 records
url: http://www.espn.com/chalk/story/_/id/25209329/los-angeles-kings-president-luc-robitaille-explains-why-gambling-revenue-affect-nhl-ticket-prices
text already downloaded, skipping...
Processing 112 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25197640/nhl-zac-dalpe-cleveland-monsters-player-world-week
text already downloaded, skipping...
Processing 113 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25195157/nhl-faq-chicago-blackhawks-firing-joel-quenneville
text already downloaded, skipping...
Processing 114 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25177678/nhl-adidas-reinvents-original-six-jerseys-ea-sports-nhl-19
text already downloaded, skipping...
Processing 115 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25155563/early-nhl-awards-race-update-hart-norris-selke-vezina-calder-adams
text already downloaded, skipping...
Processing 116 / 7289 records
url: http://www.espn.com/nhl/story/_/id/25155303/blades-steel-

HTTP status code 404 raised for url: http://www.espn.com/espn/now?nowId=21-41034239-4
Processing 181 / 7289 records
url: http://www.espn.com/espn/now?nowId=21-41034240-4
HTTP status code 404 raised for url: http://www.espn.com/espn/now?nowId=21-41034240-4
Processing 182 / 7289 records
url: http://www.espn.com/espn/now?nowId=21-41034241-4
HTTP status code 404 raised for url: http://www.espn.com/espn/now?nowId=21-41034241-4
Processing 183 / 7289 records
url: http://www.espn.com/espn/now?nowId=21-41034242-4
text already downloaded, skipping...
Processing 184 / 7289 records
url: http://www.espn.com/nhl/story/_/id/24766223/erik-karlsson-unveils-new-san-jose-sharks-jersey-surprise-appearance
text already downloaded, skipping...
Processing 185 / 7289 records
url: http://sports-ak.espn.go.com/nhl/story/_/id/24740909/nhl-ilya-kovalchuk-decision-join-los-angeles-kings-living-beverly-hills-alex-ovechkin-more?src=rss
Unhandled domain: sports-ak.espn.go.com
Processing 186 / 7289 records
url: http:/

Processing 264 / 7289 records
url: http://sports-ak.espn.go.com/nhl/story/_/id/23779853/nhl-ilya-kovalchuk-possible-free-agent-destinations-include-bruins-rangers-kings-sharks?src=rss
Unhandled domain: sports-ak.espn.go.com
Processing 265 / 7289 records
url: http://www.espn.com/nhl/story/_/id/23761125/nhl-weekly-reader-jon-taffer-vegas-golden-knights-fandom-plus-jersey-foul-week-puck-headlines-more
text already downloaded, skipping...
Processing 266 / 7289 records
url: http://kwese.espn.com/nhl/story/_/id/23761628/nhl-ranking-alex-ovechkin-top-moments-washington-capitals-won-stanley-cup
Unhandled domain: kwese.espn.com
Processing 267 / 7289 records
url: http://www.espn.com/nhl/story/_/id/23730429/washington-capitals-championship-parade-bigger-everyone-expecting-owner-ted-leonsis-says
text already downloaded, skipping...
Processing 268 / 7289 records
url: http://www.espn.com/nhl/story/_/id/23718094/washington-capitals-fans-70000-viewing-party
text already downloaded, skipping...
Process

...

Processing 7162 / 7289 records
url: http://mmqb.si.com/mmqb/2017/04/18/nfl-draft-corey-davis-western-michigan-ankle-injury-40-time
Unhandled domain: mmqb.si.com
Processing 7163 / 7289 records
url: http://mmqb.si.com/mmqb/2017/04/13/patrick-mahomes-nfl-draft-pro-day-texas-tech
Unhandled domain: mmqb.si.com
Processing 7164 / 7289 records
url: http://www.si.com/mmqb/2017/04/11/nfl-draft-jordan-willis-kansas-state-riser-first-round-deshone-kizer-brian-kelly
Processing 7165 / 7289 records
url: http://mmqb.si.com/mmqb/2017/04/04/nfl-draft-best-running-backs-2014-leonard-fournette-ezekiel-elliott-dalvin-cook-tre-mason-christian
Unhandled domain: mmqb.si.com
Processing 7166 / 7289 records
url: http://www.si.com/mmqb/2017/03/28/nfl-draft-myles-garrett-kevin-sumlin-competitiveness-attitude
Processing 7167 / 7289 records
url: http://www.si.com/mmqb/2017/03/21/nfl-mock-drafts-mel-kiper-todd-mcshay-daniel-jeremiah-deshaun-watson-pro-day-mike-williams-40-time
Processing 7168 / 7289 records
url: http

Processing 7216 / 7289 records
url: http://www.si.com/mlb/2016/10/19/chicago-cubs-nlcs-carol-haddon
Processing 7217 / 7289 records
url: http://www.si.com/mmqb/2016/10/12/nfl-draft-tj-watt-wisconsin-jj-watt-brother
Processing 7218 / 7289 records
url: http://mmqb.si.com/mmqb/2016/10/12/nfl-draft-tj-watt-wisconsin-jj-watt-brother
Unhandled domain: mmqb.si.com
Processing 7219 / 7289 records
url: http://mmqb.si.com/mmqb/2016/10/05/nfl-draft-eastern-illinois-university-mike-shanahan-sean-payton-jimmy-garoppolo-tony-romo-ryan-pace
Unhandled domain: mmqb.si.com
Processing 7220 / 7289 records
url: http://mmqb.si.com/mmqb/2016/09/28/nfl-draft-corey-davis-western-michigan-first-round
Unhandled domain: mmqb.si.com
Processing 7221 / 7289 records
url: http://www.si.com/mmqb/2016/09/28/nfl-draft-corey-davis-western-michigan-first-round
Processing 7222 / 7289 records
url: http://www.si.com/mmqb/2016/09/21/deshone-kizer-nfl-draft-notre-dame-rick-mirer-brady-quinn-tony-rice
Processing 7223 / 7289 record

Traceback (most recent call last):
  File "<ipython-input-217-f61c3f64274d>", line 48, in domain_specific_parser
    actual_text = text_parent.findAll("p")
AttributeError: 'NoneType' object has no attribute 'findAll'


Unhandled exception: None
Processing 7228 / 7289 records
url: http://mmqb.si.com/mmqb/2016/08/23/nfl-ben-utecht-book-counting-days-while-my-mind-slips-away
Unhandled domain: mmqb.si.com
Processing 7229 / 7289 records
url: http://mmqb.si.com/mmqb/2016/08/17/nfl-chicago-bears-jay-cutler
Unhandled domain: mmqb.si.com
Processing 7230 / 7289 records
url: http://mmqb.si.com/mmqb/2016/08/12/week-review
Unhandled domain: mmqb.si.com
Processing 7231 / 7289 records
url: http://mmqb.si.com/mmqb/2016/05/11/nfl-josh-norman-redskins-how-deal-happened
Unhandled domain: mmqb.si.com
Processing 7232 / 7289 records
url: http://mmqb.si.com/mmqb/2016/04/15/jarvis-landry-nfl-miami-dolphins-adam-gase-will-smith-new-orleans
Unhandled domain: mmqb.si.com
Processing 7233 / 7289 records
url: http://www.si.com/mmqb/2016/03/15/johnny-manziel-nfl-cleveland-browns
Processing 7234 / 7289 records
url: http://www.si.com/mmqb/2016/03/02/nfl-greg-olsen-carolina-panthers-tight-end-nfl-network-combine-analyst
Processing 72

Processing 7287 / 7289 records
url: http://mmqb.si.com/2013/09/20/mark-adickes-redskins-lineman-turned-rg3-surgeon/
Unhandled domain: mmqb.si.com
Processing 7288 / 7289 records
url: http://www.boston.com/sports/baseball/redsox/2013/07/24/dustin-pedroia-signs-eight-year-million-contract-extension-with-red-sox/h8k3gwwU1waooxG1L52SvK/story.html
Unhandled domain: www.boston.com


In [231]:
# Split content into 3 files
ix = df["freetext"] == ""
numpy.sum(ix)
df_content = df[numpy.logical_not(ix)]

def write_text_file(author, dataframe):
    data = dataframe[dataframe["writer"] == author]
    with open(author + ".txt","w") as f:
        for row_ix, row in data.iterrows():
            f.write("\n" + row["freetext"] + "\n")

write_text_file("greg", df_content)
write_text_file("ryan", df_content)
write_text_file("emily", df_content)

        

In [ ]:
# Create Greg-bot
greg_bot = textgenrnn()
greg_bot.config["max_length"] = 40
greg_bot.train_from_largetext_file("greg.txt", num_epochs=2, batch_size=600)



Training new model w/ 2-layer, 128-cell LSTMs
Training on 18,867,841 character sequences.
Epoch 1/2
11310/31446 [=========>....................] - ETA: 7:49:17 - loss: 1.5556